In [ ]:
import re
from decimal import Decimal

#处理嵌套的列表，字典
def clean_nesting(data):
    result=[]
    if isinstance(data,list):
        for item in data:
            result.extend(clean_nesting(item))

    elif isinstance(data,dict):
        for value in data.values():
            result.extend(clean_nesting(value))

    else:
        result.append(data)

    return result


#收集科学计数法，全角数字，半角数字
def collect_obvious_digits(lst):
    result=[]
    for item in lst:
        item_str = str(item)
        s=re.sub(r'[^0-9eE+.,\uff01-\uff5e-]','',item_str).strip()
        result.append(s)
    return result


#全角转半角
def full_to_half(lst):
    result = []  # 存放每个元素处理后的字符串
    for item in lst:
        char_list = []  # 存放当前元素的有效字符
        a = str(item)
        allowed = '.eE+-,'
        for char in a:
            if '\uff10' <= char <= '\uff19':  # 全角数字
                half_char = chr(ord(char) - 65248)
                char_list.append(half_char)
            elif '0' <= char <= '9':  # 半角数字
                char_list.append(char)
            elif char in allowed:  # 允许的特殊字符
                char_list.append(char)

        # 把当前元素的字符列表转成字符串，加到结果
        result.append(''.join(char_list))

    return result  # 返回字符串列表


#去除格式化
def clean_format(lst):
    result=[]
    for item in lst:
        s=str(item)
        if ',' in s and '.' in s:
            if s.find('.') > s.find(','):
                s=s.replace(',','')
            elif s.find('.') < s.find(','):
                s=s.replace(',','.').replace('.','')
            result.append(s)
        elif ',' in s:
            s=s.replace(',','.')
            result.append(s)
        else:
            result.append(s)
    return result

#去除重复小数点
def clean_period(lst):
    result=[]
    for item in lst:
        s = str(item)
        if s.find('.') != s.rfind('.'):
            if s.find('.') == 0 or s.find('.')== (len(s)-1):
                s=s.replace('.','')
                result.append(s)
            else:
                s=s.replace('.','',1)
                result.append(s)
        else:
            result.append(s)
    return result


#字符串转浮点数
def str_to_float(lst):
    result = []
    for item in lst:
        s = str(item).strip()
        if not s:  # 跳过空字符串
            continue

        try:
            # 先尝试普通转换
            num = float(s)
            result.append(num)
        except ValueError:
            try:
                # 再尝试Decimal（处理科学计数法更好）
                num = Decimal(s)
                result.append(float(num))
            except:
                # 转换失败，跳过
                continue
    return result


#以85为阈值过滤并进行归一化处理
def filter_under_eightyfive(lst):
    result = []

    # 归一化参数
    src_min = 80.0  # 原始数据最小值
    src_max = 120.0  # 原始数据最大值
    dst_min = 0.01  # 目标最小值
    dst_max = 1.99  # 目标最大值

    # 计算缩放因子（提前算好，避免重复计算）
    scale = (dst_max - dst_min) / (src_max - src_min)  # 0.0495

    for digit in lst:
        # 只处理≥80的数值（题目要求）
        if digit >= 80:
            # 科学归一化公式：y = (x - src_min) * scale + dst_min
            normalized = (digit - src_min) * scale + dst_min

            # 确保在目标范围内（处理超出120的值）
            if normalized <dst_min:
                normalized = dst_min
            elif normalized >dst_max:
                normalized = dst_max

            # 保留两位小数（题目要求0.xx-1.xx）
            result.append(round(normalized, 2))

    return result


def process_data(raw_data):
    result_1=clean_nesting(raw_data)
    result_2=collect_obvious_digits(result_1)
    result_3=full_to_half(result_2)
    result_4=clean_format(result_3)
    result_5=clean_period(result_4)
    result_6=str_to_float(result_5)
    result_7=filter_under_eightyfive(result_6)
    return result_7



def main():
    raw_data=[
  ["85", "120", "ERROR", "   99   ", "79.99999999999999", "80.000000000001"],
  ["\u0000", "\u001F", " ", "\t\n\r"],
  ["１２０", "８０.５", "７９", "100.0"],
  ["0x50", "0b1010000", "0o120", "80"],
  ["88,5", "1.200,50", "1,200.00"],
  ["1e2", "-1e2", "5e-1", "999999999999999999999999999999.9", "1.7976931348623157e+308"],
  [True, False, "NaN", "Infinity", "-Infinity", "undefined", "null"],
  ["80%", "120V", "$100", "95.5MHz", "核心温度:90度"],
  ["8\u200B5", "1\u20280\u20295", "9\u00A02"],
  ["..90", "90..", "9.0.5", "100.0.0"],
  ["-0", "+85", "--85", "- 100"],
  ["SGVsbG8=", "0xFF", "Error_Code_404"],
  ["82.0", "81.999", "80.0000000000", "79.9999"],
  [],
  [{"status": "online", "value": "105"}],
  ["95", ["110", "120"], [["130"]]],
  ["99.9", "   ", " @#￥%……&* ", "运转正常", "核心过载"],
  ["12345678901234567890", "0.0000000000000000000000085"],
  ["🔥100", "85abc", "abc85", "1a2b3c"],
  ["81", "82", "83", "84", "85", "86", "87", "88", "89", "90", "91", "92", "93", "94", "95", "96", "97", "98", "99", "100"],
  ["79.9", "79.99", "79.999", "79.9999", "79.99999"],
  ["\u4e00\u4e8c\u4e09", "100\u20ac"],
  ["-9999", "9999", "0.00001"],
  ["80.0000000000001", "80.0000000000002", "80.0000000000003"],
  [" ", "  ", "   ", "    ", "     "],
  ["101", "102", "103"],
  ["120", "120.0", "120.000000", "000120"],
  ["85.5.5", "85..5", "85. 5", "8 5.5"],
  ["[100]", "(100)", "{100}"],
  ["1.000.000", "80,00"],
  ["95", "96", "97", "ERROR", "WARNING", "CRITICAL", "FATAL"],
  ["0", "1", "2", "3", "78", "79", "80"],
  ["120", "\r", "121", "\n", "122"],
  ["\uD83D\uDE80", "\uD83D\uDCAF"],
  ["110", "NULL", "None", "n/a", "N/A", "NaN"],
  ["80.1", "80.01", "80.001", "80.0001"],
  ["125", " ", "126", " ", "127"],
  ["1e1", "1e+1", "1e01", "10.0"],
  ["90", "91", "92", "93", "94", "95", "96", "97", "98", "99", "100", "101", "102"],
  ["0.85e2", "0.12e3"],
  ["-0.0", "0.0", ".0", "0."],
  ["105", " ", "", "   ", "106"],
  ["92", "fault", "93", "94", "95", "96"],
  ["88", "88", "88", "88", "88"],
  ["150", "1.5e2", "150.000"],
  [" \t\n\r\f85\f\r\n\t "],
  ["000085", "000000000000120"],
  ["99.999", "100.001", "100.000"],
  ["能源核心最后一条数据", "100", "End"]
]

    def determine_overload(lst):
        if not lst:  # 空列表检查
            print("没有有效数据")
            return

        # 计算平均值
        avg = sum(lst) / len(lst)

        # 整体判断
        if avg >= 1.0:
            print(f'平均值{avg:.2f}:核心过载')
        else:
            print(f'平均值{avg:.2f}:运转正常')

    determine_overload(process_data(raw_data))

    
    import os
    os.system("pause")
    # input("按回车键退出...")


if __name__ == "__main__":
    main()